# Como calcular a inflação implícita e decidir o melhor Tesouro Direto

## 1 - Instalar libs

In [42]:
!pip install requests beautifulsoup4 pandas tabulate
%pdb on

Automatic pdb calling has been turned ON


In [43]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tabulate import tabulate
import re
from datetime import datetime
import json
import logging
from typing import Dict, List, Tuple, Optional, Any


# Configuração de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Constantes
TESOURO_URL = "https://www.tesourodireto.com.br/titulos/precos-e-taxas.htm"
TESOURO_API_URL = "https://www.tesourodireto.com.br/json/br/com/b3/tesourodireto/service/api/treasurybondsinfo.json"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

## Definindo modelos

In [44]:
# Tipos de títulos
class TipoTitulo:
    PREFIXADO = "Prefixado"
    IPCA = "IPCA"

# Modelos de dados
class Titulo:
    def __init__(self, nome: str, vencimento: str, vencimento_data: datetime, rentabilidade: float):
        self.nome = nome
        self.vencimento = vencimento
        self.vencimento_data = vencimento_data
        self.rentabilidade = rentabilidade

    def to_dict(self) -> Dict[str, Any]:
        return {
            "nome": self.nome,
            "vencimento": self.vencimento,
            "vencimento_data": self.vencimento_data,
            "rentabilidade": self.rentabilidade
        }

class ResultadoComparativo:
    def __init__(
        self,
        titulo_prefixado: Titulo,
        titulo_ipca: Titulo,
        inflacao_implicita: float
    ):
        self.titulo_prefixado = titulo_prefixado
        self.titulo_ipca = titulo_ipca
        self.inflacao_implicita = inflacao_implicita

        # Determinar recomendação com base na inflação implícita
        if inflacao_implicita > 5:  # Limite conservador para considerar inflação alta
            self.recomendacao = f"Se você acredita que a inflação será MAIOR que {inflacao_implicita:.2f}%, escolha IPCA+. Caso contrário, escolha Prefixado."
        else:
            self.recomendacao = f"Se você acredita que a inflação será MENOR que {inflacao_implicita:.2f}%, escolha Prefixado. Caso contrário, escolha IPCA+."

    def to_dict(self) -> Dict[str, Any]:
        return {
            "Título Prefixado": self.titulo_prefixado.nome,
            "Vencimento Prefixado": self.titulo_prefixado.vencimento,
            "Taxa Prefixado (%)": self.titulo_prefixado.rentabilidade,
            "Título IPCA+": self.titulo_ipca.nome,
            "Vencimento IPCA+": self.titulo_ipca.vencimento,
            "Taxa IPCA+ (%)": self.titulo_ipca.rentabilidade,
            "Inflação Implícita (%)": round(self.inflacao_implicita, 2),
            "Recomendação": self.recomendacao
        }


## 2 -Acessa o site do Tesouro Direto para obter os dados atualizados dos títulos
## 3 Para cada título prefixado, encontra o título IPCA+ com vencimento mais próximo

In [45]:
class TesouroDiretoExtractor:
    """Classe para extrair dados do Tesouro Direto utilizando vários métodos"""

    def __init__(self):
        self.tesouro_url = "https://www.tesourodireto.com.br/titulos/precos-e-taxas.htm"
        self.tesouro_api_url = "https://www.tesourodireto.com.br/json/br/com/b3/tesourodireto/service/api/treasurybondsinfo.json"
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        self.titulos_prefixados = []
        self.titulos_ipca = []

    def _fazer_requisicao(self, url: str) -> Optional[requests.Response]:
        """Faz uma requisição HTTP para a URL especificada"""
        try:
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            return response
        except requests.exceptions.RequestException as e:
            logger.error(f"Erro ao acessar a URL {url}: {e}")
            return None

    def _extrair_titulos_do_json(self, script_content: str) -> None:
        """Extrai informações de títulos de um script contendo JSON"""
        try:
            json_match = re.search(r'window\.TD\.titulos\s*=\s*(\[.*?\]);', script_content, re.DOTALL)
            if not json_match:
                return

            json_data = json.loads(json_match.group(1))

            for titulo_data in json_data:
                nome = titulo_data.get('nome', '')
                vencimento = titulo_data.get('vencimento', '')

                try:
                    rentabilidade = float(titulo_data.get('rentabilidade', '0').replace('%', '').replace(',', '.').strip())
                except (ValueError, AttributeError):
                    rentabilidade = 0

                if not vencimento:
                    continue

                try:
                    vencimento_data = datetime.strptime(vencimento, '%d/%m/%Y')
                except ValueError:
                    try:
                        vencimento_data = datetime(int(vencimento), 1, 1)
                    except ValueError:
                        continue

                titulo = Titulo(nome, vencimento, vencimento_data, rentabilidade)

                if TipoTitulo.PREFIXADO in nome:
                    self.titulos_prefixados.append(titulo)
                elif TipoTitulo.IPCA in nome:
                    self.titulos_ipca.append(titulo)

        except Exception as e:
            logger.error(f"Erro ao extrair dados do script: {e}")

    def _extrair_titulos_de_divs(self, soup: BeautifulSoup) -> None:
        """Extrai informações de títulos a partir das divs da página"""
        div_titulos = soup.find_all('div', class_=lambda c: c and ('card-title' in c or 'titulo' in c or 'tesouro' in c))

        for div in div_titulos:
            try:
                nome_titulo = div.get_text().strip()
                taxa_divs = div.find_next_siblings('div') or div.find_all('div')

                for taxa_div in taxa_divs:
                    taxa_text = taxa_div.get_text().strip()
                    taxa_match = re.search(r'(\d+[,.]\d+)\s*%', taxa_text)

                    if not taxa_match:
                        continue

                    rentabilidade = float(taxa_match.group(1).replace(',', '.'))
                    vencimento_match = re.search(r'(\d{2}/\d{2}/\d{4}|20\d{2})', nome_titulo)

                    if not vencimento_match:
                        continue

                    vencimento = vencimento_match.group(1)

                    try:
                        if len(vencimento) == 4:  # Apenas o ano
                            vencimento_data = datetime(int(vencimento), 1, 1)
                        else:
                            vencimento_data = datetime.strptime(vencimento, '%d/%m/%Y')
                    except ValueError:
                        continue

                    titulo = Titulo(nome_titulo, vencimento, vencimento_data, rentabilidade)

                    if TipoTitulo.PREFIXADO in nome_titulo:
                        self.titulos_prefixados.append(titulo)
                    elif TipoTitulo.IPCA in nome_titulo:
                        self.titulos_ipca.append(titulo)
                    break
            except Exception as e:
                logger.warning(f"Erro ao processar div: {e}")

    def _extrair_titulos_de_texto(self, soup: BeautifulSoup) -> None:
        """Extrai informações de títulos a partir do texto da página"""
        text = soup.get_text()

        # Padrões para extração de texto
        patterns = [
            (TipoTitulo.PREFIXADO, re.compile(r'(Tesouro\s+Prefixado\s+.*?20\d{2})[^\d]+([\d,]+%)')),
            (TipoTitulo.IPCA, re.compile(r'(Tesouro\s+IPCA\+\s+.*?20\d{2})[^\d]+([\d,]+%)'))
        ]

        for tipo, pattern in patterns:
            for match in pattern.finditer(text):
                nome = match.group(1).strip()
                taxa_str = match.group(2).strip().replace('%', '').replace(',', '.')

                try:
                    taxa = float(taxa_str)
                except ValueError:
                    continue

                vencimento_match = re.search(r'20\d{2}', nome)
                if not vencimento_match:
                    continue

                vencimento = vencimento_match.group(0)
                vencimento_data = datetime(int(vencimento), 1, 1)

                titulo = Titulo(nome, vencimento, vencimento_data, taxa)

                if tipo == TipoTitulo.PREFIXADO:
                    self.titulos_prefixados.append(titulo)
                elif tipo == TipoTitulo.IPCA:
                    self.titulos_ipca.append(titulo)

    def _extrair_titulos_da_api(self) -> None:
        """Extrai informações de títulos diretamente da API do Tesouro Direto"""
        response = self._fazer_requisicao(self.tesouro_api_url)
        if not response:
            return

        try:
            api_data = response.json()

            if 'response' not in api_data or 'TrsrBdTradgList' not in api_data['response']:
                return

            for item in api_data['response']['TrsrBdTradgList']:
                nome = item.get('TrsrBd', {}).get('nm', '')
                vencimento = item.get('TrsrBd', {}).get('mtrtyDt', '')
                taxa_str = item.get('anulInvstmtRate', '').replace(',', '.')

                if not nome or not vencimento or not taxa_str:
                    continue

                try:
                    taxa = float(taxa_str)
                    vencimento_data = datetime.strptime(vencimento, '%Y-%m-%d')
                    vencimento_str = vencimento_data.strftime('%d/%m/%Y')

                    titulo = Titulo(nome, vencimento_str, vencimento_data, taxa)

                    if TipoTitulo.PREFIXADO in nome:
                        self.titulos_prefixados.append(titulo)
                    elif TipoTitulo.IPCA in nome:
                        self.titulos_ipca.append(titulo)
                except (ValueError, TypeError) as e:
                    logger.warning(f"Erro ao processar item da API: {e}")
                    continue
        except Exception as e:
            logger.error(f"Erro ao acessar API: {e}")

    def extrair_dados(self, usar_dados_exemplo: bool = False) -> Tuple[List[Dict], List[Dict]]:
        """
        Obtém dados do Tesouro Direto usando várias estratégias em cascata

        Args:
            usar_dados_exemplo: Se True, ignora extração da web e usa dados de exemplo

        Returns:
            Tupla com duas listas de dicionários: títulos prefixados e títulos IPCA+
        """
        # Limpar dados anteriores
        self.titulos_prefixados = []
        self.titulos_ipca = []

        logger.info("Iniciando extração de dados do Tesouro Direto")

        # Tenta fazer requisição para a página principal
        response = self._fazer_requisicao(self.tesouro_url)
        if not response:
            logger.error("Não foi possível acessar a página do Tesouro Direto")
            return self._obter_resultado()

        # Parsear o conteúdo HTML
        soup = BeautifulSoup(response.content, 'html.parser')

        # Estratégia 1: Procurar script com dados JSON
        logger.info("Tentando extração via JSON embutido no script")
        script_tags = soup.find_all('script')
        for script in script_tags:
            if not script.string or "window.TD" not in script.string:
                continue
            self._extrair_titulos_do_json(script.string)
            break

        # Se a primeira estratégia não funcionar, tenta a segunda
        if not self.titulos_prefixados and not self.titulos_ipca:
            logger.info("Tentando extração via divs da página")
            self._extrair_titulos_de_divs(soup)

        # Se a segunda estratégia não funcionar, tenta a terceira
        if not self.titulos_prefixados and not self.titulos_ipca:
            logger.info("Tentando extração via texto completo da página")
            self._extrair_titulos_de_texto(soup)

        # Se nenhuma das estratégias anteriores funcionou, tenta a API
        if not self.titulos_prefixados and not self.titulos_ipca:
            logger.info("Tentando acesso direto à API")
            self._extrair_titulos_da_api()

        logger.info(f"Extração concluída: {len(self.titulos_prefixados)} títulos prefixados e {len(self.titulos_ipca)} títulos IPCA+")
        return self._obter_resultado()

    def _obter_resultado(self) -> Tuple[List[Dict], List[Dict]]:
        """Retorna os resultados como listas de dicionários"""
        return (
            [titulo.to_dict() for titulo in self.titulos_prefixados],
            [titulo.to_dict() for titulo in self.titulos_ipca]
        )

## 4 Realiza o cálculo da inflação implícita conforme a fórmula que você especificou:
    - Soma 1 à taxa do título prefixado
    - Soma 1 à taxa do título IPCA+
    - Divide o primeiro resultado pelo segundo
    - Subtrai 1 do resultado e converte para percentual

In [46]:
# Funções de cálculo e análise
def calcular_comparativo(titulos_prefixados: List[Dict], titulos_ipca: List[Dict]) -> List[Dict]:
    """Calcula o comparativo entre títulos prefixados e IPCA+"""
    logger.info("Calculando comparativos entre títulos")

    resultados = []

    if not titulos_prefixados or not titulos_ipca:
        logger.warning("Não há dados suficientes para calcular o comparativo")
        return resultados

    # Ordenar títulos por data de vencimento
    titulos_prefixados.sort(key=lambda x: x["vencimento_data"])
    titulos_ipca.sort(key=lambda x: x["vencimento_data"])

    # Para cada título prefixado, encontrar o IPCA+ mais próximo em termos de vencimento
    for prefixado in titulos_prefixados:
        # Encontrar o IPCA+ mais próximo
        ipca_mais_proximo = None
        diferenca_minima = float('inf')

        for ipca in titulos_ipca:
            diferenca = abs((prefixado["vencimento_data"] - ipca["vencimento_data"]).days)
            if diferenca < diferenca_minima:
                diferenca_minima = diferenca
                ipca_mais_proximo = ipca

        if ipca_mais_proximo:
            # Calcular a inflação implícita
            prefixado_valor = 1 + (prefixado["rentabilidade"] / 100)
            ipca_valor = 1 + (ipca_mais_proximo["rentabilidade"] / 100)
            resultado = (prefixado_valor / ipca_valor) - 1
            resultado_percentual = resultado * 100

            # Criar objeto de resultado e converter para dicionário
            titulo_prefixado = Titulo(
                prefixado["nome"],
                prefixado["vencimento"],
                prefixado["vencimento_data"],
                prefixado["rentabilidade"]
            )

            titulo_ipca = Titulo(
                ipca_mais_proximo["nome"],
                ipca_mais_proximo["vencimento"],
                ipca_mais_proximo["vencimento_data"],
                ipca_mais_proximo["rentabilidade"]
            )

            comparativo = ResultadoComparativo(
                titulo_prefixado,
                titulo_ipca,
                resultado_percentual
            )

            resultados.append(comparativo.to_dict())

    return resultados

## 5 Apresenta os resultados em uma tabela com as seguintes colunas:

In [47]:
# Funções de exibição e relatório
def exibir_resultados(resultados: List[Dict]) -> Optional[pd.DataFrame]:
    """Exibe os resultados da análise de inflação implícita"""
    if not resultados:
        logger.warning("Não foi possível calcular os resultados devido à falta de dados")
        return None

    # Converter para DataFrame
    df = pd.DataFrame(resultados)

    # Exibir cabeçalho
    print("\n=== ANÁLISE DE INFLAÇÃO IMPLÍCITA NO TESOURO DIRETO ===\n")

    # Criar uma versão simplificada para exibição
    df_display = df[["Título Prefixado", "Taxa Prefixado (%)",
                     "Título IPCA+", "Taxa IPCA+ (%)",
                     "Inflação Implícita (%)"]]

    print(tabulate(df_display, headers='keys', tablefmt='grid', showindex=False))

    # Exibir explicação e recomendação para cada par de títulos
    print("\n=== INTERPRETAÇÃO E RECOMENDAÇÃO ===\n")

    for i, row in enumerate(resultados):
        print(f"Par {i+1}: {row['Título Prefixado']} vs {row['Título IPCA+']}")
        print(f"Inflação Implícita: {row['Inflação Implícita (%)']:.2f}%")
        print("O que significa: Este é o valor médio anual do IPCA até o vencimento")
        print("que o mercado está prevendo implicitamente.")
        print(f"Recomendação: {row['Recomendação']}")
        print("-" * 80)

    # Explicação adicional
    print("\nEXPLICAÇÃO DETALHADA:")
    print("""
A inflação implícita representa quanto o mercado acha que será o IPCA médio anual até o vencimento do título.

ESTRATÉGIA DE INVESTIMENTO:
- Se você acredita que a inflação média anual será MAIOR que a inflação implícita,
  o melhor negócio é investir no título indexado ao IPCA.
- Se você acredita que a inflação média anual será MENOR que a inflação implícita,
  o título prefixado é a escolha mais rentável.

Esta estratégia vale para qualquer título de renda fixa, desde que tenham o mesmo prazo e emissor.
Isso garante que você está comparando ativos com níveis de risco similares.
    """)

    # Retornar também o DataFrame para possível uso posterior
    return df
# Função principal
def main() -> None:
    """Função principal do programa"""
    extractor = TesouroDiretoExtractor()


    try:
        # Obter dados do Tesouro Direto
        titulos_prefixados, titulos_ipca = extractor.extrair_dados()

        # Se não conseguiu obter dados, usar dados de exemplo
        if not titulos_prefixados or not titulos_ipca:
            logger.warning("Não foi possível obter dados reais. Usando dados de exemplo.")

        # Calcular o comparativo
        resultados = calcular_comparativo(titulos_prefixados, titulos_ipca)

        # Exibir resultados
        df_resultados = exibir_resultados(resultados)

    except Exception as e:
        logger.error(f"Erro inesperado na execução do programa: {e}")

if __name__ == "__main__":
    main()

2025-02-28 17:06:03,071 - INFO - Iniciando extração de dados do Tesouro Direto
2025-02-28 17:06:03,211 - INFO - Tentando extração via JSON embutido no script
2025-02-28 17:06:03,212 - INFO - Tentando extração via divs da página
2025-02-28 17:06:03,213 - INFO - Tentando extração via texto completo da página
2025-02-28 17:06:03,213 - INFO - Tentando acesso direto à API
2025-02-28 17:06:03,323 - INFO - Extração concluída: 0 títulos prefixados e 0 títulos IPCA+
2025-02-28 17:06:03,324 - WARNING - Não foi possível obter dados reais. Usando dados de exemplo.
2025-02-28 17:06:03,324 - INFO - Calculando comparativos entre títulos
2025-02-28 17:06:03,325 - WARNING - Não há dados suficientes para calcular o comparativo
2025-02-28 17:06:03,325 - WARNING - Não foi possível calcular os resultados devido à falta de dados
